This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [8]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'alpha' : 0.15, 
    'sample_size' : 2000,
    'edge_modification_ratio' : 0.2,
    'feature_mask_ratio' : 0.2,
    'beta' : 0.6,
}

generator_config = {
    'num_clusters' : 7,
}

pretext_task = MERIT

In [9]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [10]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_task=pretext_task)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: User

([10.279911041259766,
  10.256790161132812,
  10.219374656677246,
  10.160697937011719,
  10.102499008178711,
  10.014968872070312,
  9.856452941894531,
  9.807066917419434,
  9.680686950683594,
  9.49393367767334,
  9.409756660461426,
  9.258402824401855,
  9.11376667022705,
  8.983085632324219,
  8.900452613830566,
  8.81494426727295,
  8.740604400634766,
  8.647851943969727,
  8.568242073059082,
  8.570302963256836,
  8.417478561401367,
  8.398058891296387,
  8.45672607421875,
  8.394245147705078,
  8.303811073303223,
  8.30394172668457,
  8.310691833496094,
  8.339522361755371,
  8.312296867370605,
  8.20773983001709,
  8.300609588623047,
  8.237146377563477,
  8.192801475524902,
  8.134872436523438,
  8.276854515075684,
  8.164108276367188,
  8.190421104431152,
  8.206107139587402,
  8.197702407836914,
  8.161581039428711,
  8.131065368652344,
  8.151775360107422,
  8.068318367004395,
  8.104947090148926,
  8.071589469909668,
  8.14771842956543,
  8.058927536010742,
  8.1407279968